In [1]:
# N2 - CELDA 1: Setup (reusar Spark + sfOptions del Notebook 1)

import os
from pyspark.sql import SparkSession

try:
    spark.stop()
except:
    pass

spark = SparkSession.builder.getOrCreate()

# Reusa sfOptions del N1; si no existe, vuelve a definirlo rápido:
if "sfOptions" not in globals():
    sfOptions = {
        "sfURL": os.getenv("SNOWFLAKE_HOST"),
        "sfUser": os.getenv("SNOWFLAKE_USER"),
        "sfPassword": os.getenv("SNOWFLAKE_PASSWORD"),
        "sfDatabase": os.getenv("SNOWFLAKE_DATABASE"),
        "sfSchema": os.getenv("SNOWFLAKE_SCHEMA_RAW"),
        "sfWarehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
        "sfRole": os.getenv("SNOWFLAKE_ROLE"),
        "sfAccount": os.getenv("SNOWFLAKE_HOST").replace(".snowflakecomputing.com",""),
    }

# ping rápido
ping = (spark.read.format("snowflake")
        .options(**sfOptions)
        .option("query", "SELECT CURRENT_ACCOUNT() acct, CURRENT_DATABASE() db, CURRENT_SCHEMA() sch, CURRENT_WAREHOUSE() wh, CURRENT_ROLE() role")
        .load())
ping.show(truncate=False)
print("Setup N2 OK ✅")


+--------+--------+---+-----+--------+
|ACCT    |DB      |SCH|WH   |ROLE    |
+--------+--------+---+-----+--------+
|XPC24435|DM_PSET3|RAW|WH_DM|SYSADMIN|
+--------+--------+---+-----+--------+

Setup N2 OK ✅


In [8]:
pip install snowflake-connector-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 8.8 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 12.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
# N2 - CELDA 2 (FIX definitivo): inventario de RAW en la DB correcta

DB = os.getenv("SNOWFLAKE_DATABASE")  # debería ser DM_PSET3

q_y = (
    "SELECT table_name,"
    " TRY_TO_NUMBER(REGEXP_SUBSTR(table_name, 'TRIPS_YELLOW_(\\\\d{4})_', 1, 1, 'e', 1)) AS yr,"
    " TRY_TO_NUMBER(REGEXP_SUBSTR(table_name, 'TRIPS_YELLOW_\\\\d{4}_(\\\\d{2})', 1, 1, 'e', 1)) AS mo"
    " FROM " + DB + ".INFORMATION_SCHEMA.TABLES"
    " WHERE table_schema='RAW' AND table_name ILIKE 'TRIPS_YELLOW_%'"
    "   AND TRY_TO_NUMBER(REGEXP_SUBSTR(table_name, 'TRIPS_YELLOW_(\\\\d{4})_', 1, 1, 'e', 1)) IS NOT NULL"
    "   AND TRY_TO_NUMBER(REGEXP_SUBSTR(table_name, 'TRIPS_YELLOW_\\\\d{4}_(\\\\d{2})', 1, 1, 'e', 1)) IS NOT NULL"
    " ORDER BY yr, mo"
)

q_g = (
    "SELECT table_name,"
    " TRY_TO_NUMBER(REGEXP_SUBSTR(table_name, 'TRIPS_GREEN_(\\\\d{4})_', 1, 1, 'e', 1)) AS yr,"
    " TRY_TO_NUMBER(REGEXP_SUBSTR(table_name, 'TRIPS_GREEN_\\\\d{4}_(\\\\d{2})', 1, 1, 'e', 1)) AS mo"
    " FROM " + DB + ".INFORMATION_SCHEMA.TABLES"
    " WHERE table_schema='RAW' AND table_name ILIKE 'TRIPS_GREEN_%'"
    "   AND TRY_TO_NUMBER(REGEXP_SUBSTR(table_name, 'TRIPS_GREEN_(\\\\d{4})_', 1, 1, 'e', 1)) IS NOT NULL"
    "   AND TRY_TO_NUMBER(REGEXP_SUBSTR(table_name, 'TRIPS_GREEN_\\\\d{4}_(\\\\d{2})', 1, 1, 'e', 1)) IS NOT NULL"
    " ORDER BY yr, mo"
)

df_y = (spark.read.format("snowflake").options(**sfOptions).option("query", q_y).load())
df_g = (spark.read.format("snowflake").options(**sfOptions).option("query", q_g).load())

print("YELLOW en RAW (primeras 5):")
df_y.show(5, truncate=False)
print("GREEN en RAW (primeras 5):")
df_g.show(5, truncate=False)

tables_y = [(r["TABLE_NAME"], int(r["YR"]), int(r["MO"])) for r in df_y.collect()]
tables_g = [(r["TABLE_NAME"], int(r["YR"]), int(r["MO"])) for r in df_g.collect()]

print(f"Total YELLOW: {len(tables_y)}  |  Total GREEN: {len(tables_g)}")


YELLOW en RAW (primeras 5):
+--------------------+----+---+
|TABLE_NAME          |YR  |MO |
+--------------------+----+---+
|TRIPS_YELLOW_2015_01|2015|1  |
|TRIPS_YELLOW_2015_02|2015|2  |
|TRIPS_YELLOW_2015_03|2015|3  |
|TRIPS_YELLOW_2015_04|2015|4  |
|TRIPS_YELLOW_2015_05|2015|5  |
+--------------------+----+---+
only showing top 5 rows

GREEN en RAW (primeras 5):
+-------------------+----+---+
|TABLE_NAME         |YR  |MO |
+-------------------+----+---+
|TRIPS_GREEN_2015_01|2015|1  |
|TRIPS_GREEN_2015_02|2015|2  |
|TRIPS_GREEN_2015_03|2015|3  |
|TRIPS_GREEN_2015_04|2015|4  |
|TRIPS_GREEN_2015_05|2015|5  |
+-------------------+----+---+
only showing top 5 rows

Total YELLOW: 128  |  Total GREEN: 129


In [5]:
# N2 - CELDA 3: generar SQL de la VIEW RAW.TRIPS_ALL y guardarlo en un .sql

from pathlib import Path

def select_clause(service: str, table_name: str, year: int, month: int) -> str:
    """
    SELECT normalizado por servicio para uniformar columnas.
    - Convierte tpep_/lpep_ a pickup/dropoff.
    - COALESCE y CAST para columnas faltantes o con nombres alternativos.
    - Agrega service_type, year, month.
    """
    if service == "yellow":
        pickup, dropoff = "tpep_pickup_datetime", "tpep_dropoff_datetime"
    else:
        pickup, dropoff = "lpep_pickup_datetime", "lpep_dropoff_datetime"

    return f"""
SELECT
  '{service}' AS service_type,
  {pickup}::timestamp AS pickup_datetime,
  {dropoff}::timestamp AS dropoff_datetime,
  COALESCE(passenger_count, NULL)::number        AS passenger_count,
  COALESCE(trip_distance, NULL)::float          AS trip_distance,
  COALESCE(PULocationID, NULL)::number          AS pu_location_id,
  COALESCE(DOLocationID, NULL)::number          AS do_location_id,
  COALESCE(fare_amount, NULL)::float            AS fare_amount,
  COALESCE(tip_amount, NULL)::float             AS tip_amount,
  COALESCE(tolls_amount, NULL)::float           AS tolls_amount,
  COALESCE(mta_tax, NULL)::float                AS mta_tax,
  COALESCE(improvement_surcharge, NULL)::float  AS improvement_surcharge,
  COALESCE(congestion_surcharge, NULL)::float   AS congestion_surcharge,
  COALESCE(total_amount, NULL)::float           AS total_amount,
  COALESCE(payment_type, NULL)::varchar         AS payment_type,
  COALESCE(vendorid, vendor_id)::varchar        AS vendor_id,
  COALESCE(ratecodeid, rate_code_id)::varchar   AS rate_code_id,
  COALESCE(store_and_fwd_flag, NULL)::varchar   AS store_and_fwd_flag,
  {year}::number  AS year,
  {month}::number AS month
FROM RAW.{table_name}
""".strip()

parts = []
for t, y, m in tables_y:
    parts.append(select_clause("yellow", t, y, m))
for t, y, m in tables_g:
    parts.append(select_clause("green", t, y, m))

sql_view = "CREATE OR REPLACE VIEW RAW.TRIPS_ALL AS\n" + "\nUNION ALL\n".join(parts) + ";\n"

# Guardar a archivo para pegar en Snowsight
out = Path("create_raw_trips_all.sql")
out.write_text(sql_view, encoding="utf-8")

print(f"✅ SQL generado con {len(parts)} SELECTs (UNION ALL).")
print(f"Archivo guardado en: {out.resolve()}")
print("\nVista previa (primeros 40 líneas):")
print("\n".join(sql_view.splitlines()[:40]))


✅ SQL generado con 257 SELECTs (UNION ALL).
Archivo guardado en: /home/jovyan/work/create_raw_trips_all.sql

Vista previa (primeros 40 líneas):
CREATE OR REPLACE VIEW RAW.TRIPS_ALL AS
SELECT
  'yellow' AS service_type,
  tpep_pickup_datetime::timestamp AS pickup_datetime,
  tpep_dropoff_datetime::timestamp AS dropoff_datetime,
  COALESCE(passenger_count, NULL)::number        AS passenger_count,
  COALESCE(trip_distance, NULL)::float          AS trip_distance,
  COALESCE(PULocationID, NULL)::number          AS pu_location_id,
  COALESCE(DOLocationID, NULL)::number          AS do_location_id,
  COALESCE(fare_amount, NULL)::float            AS fare_amount,
  COALESCE(tip_amount, NULL)::float             AS tip_amount,
  COALESCE(tolls_amount, NULL)::float           AS tolls_amount,
  COALESCE(mta_tax, NULL)::float                AS mta_tax,
  COALESCE(improvement_surcharge, NULL)::float  AS improvement_surcharge,
  COALESCE(congestion_surcharge, NULL)::float   AS congestion_surcharge,
  C

In [15]:
# N2 - CELDA 3C (versión robusta y simple con SPLIT_PART):
# Genera SQL de CREATE VIEW RAW.TRIPS_ALL detectando columnas por tabla
# Requiere: pip install snowflake-connector-python

import os, snowflake.connector
from pathlib import Path

DB  = os.getenv("SNOWFLAKE_DATABASE")
SCH = "RAW"

# 1) Conexión al diccionario de Snowflake
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_HOST").replace(".snowflakecomputing.com",""),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=DB,
    schema=SCH,
    role=os.getenv("SNOWFLAKE_ROLE"),
    client_session_keep_alive=True,
)
cur = conn.cursor()

# 2) Listado de tablas mensuales usando SPLIT_PART (más robusto que regex)
def fetch_tables(prefix):
    q = f"""
      SELECT *
      FROM (
        SELECT
          table_name,
          TRY_TO_NUMBER(SPLIT_PART(table_name, '_', 3)) AS yr,
          TRY_TO_NUMBER(SPLIT_PART(table_name, '_', 4)) AS mo
        FROM {DB}.INFORMATION_SCHEMA.TABLES
        WHERE table_schema='{SCH}'
          AND table_name ILIKE 'TRIPS_{prefix}_%'
      )
      WHERE yr IS NOT NULL AND mo IS NOT NULL
      ORDER BY yr, mo
    """
    cur.execute(q)
    return [(r[0], int(r[1]), int(r[2])) for r in cur.fetchall()]

tables_y = fetch_tables("YELLOW")
tables_g = fetch_tables("GREEN")

print(f"Tablas detectadas → YELLOW: {len(tables_y)} | GREEN: {len(tables_g)}")

# 3) Columnas existentes por tabla (en mayúsculas)
def fetch_cols(table_name):
    cur.execute(f"""
      SELECT UPPER(column_name)
      FROM {DB}.INFORMATION_SCHEMA.COLUMNS
      WHERE table_schema='{SCH}' AND table_name='{table_name}'
    """)
    return {r[0] for r in cur.fetchall()}

# 4) Helper: toma la 1ra columna disponible; si ninguna existe, devuelve NULL::tipo AS alias
def pick(cols_set, cand_list, cast_type, alias):
    for c in cand_list:
        if c in cols_set:
            return f"{c}::{cast_type} AS {alias}"
    return f"NULL::{cast_type} AS {alias}"

def select_clause(service, table_name, year, month):
    cols = fetch_cols(table_name)

    # columnas datetime (varían por servicio / años)
    pickup_col_opts  = ["TPEP_PICKUP_DATETIME","LPEP_PICKUP_DATETIME","PICKUP_DATETIME"]
    dropoff_col_opts = ["TPEP_DROPOFF_DATETIME","LPEP_DROPOFF_DATETIME","DROPOFF_DATETIME"]

    # mapeo de variantes por versión/año de schema
    mapping = [
        (["PASSENGER_COUNT"],               "number", "passenger_count"),
        (["TRIP_DISTANCE"],                 "float",  "trip_distance"),
        (["PULOCATIONID","PU_LOCATION_ID"], "number", "pu_location_id"),
        (["DOLOCATIONID","DO_LOCATION_ID"], "number", "do_location_id"),
        (["FARE_AMOUNT"],                   "float",  "fare_amount"),
        (["TIP_AMOUNT"],                    "float",  "tip_amount"),
        (["TOLLS_AMOUNT"],                  "float",  "tolls_amount"),
        (["MTA_TAX"],                       "float",  "mta_tax"),
        (["IMPROVEMENT_SURCHARGE"],         "float",  "improvement_surcharge"),
        (["CONGESTION_SURCHARGE"],          "float",  "congestion_surcharge"),
        (["TOTAL_AMOUNT"],                  "float",  "total_amount"),
        (["PAYMENT_TYPE"],                  "varchar","payment_type"),
        (["VENDORID","VENDOR_ID"],          "varchar","vendor_id"),
        (["RATECODEID","RATE_CODE_ID"],     "varchar","rate_code_id"),
        (["STORE_AND_FWD_FLAG"],            "varchar","store_and_fwd_flag"),
    ]

    parts = [
        "SELECT",
        f"  '{service.lower()}' AS service_type,",
        "  " + pick(cols, pickup_col_opts,  "timestamp", "pickup_datetime") + ",",
        "  " + pick(cols, dropoff_col_opts, "timestamp", "dropoff_datetime") + ",",
    ]
    for cand_list, tp, alias in mapping:
        parts.append("  " + pick(cols, cand_list, tp, alias) + ",")

    parts += [
        f"  {year}::number  AS year,",
        f"  {month}::number AS month",
        f"FROM {SCH}.{table_name}"
    ]
    return "\n".join(parts)

# 5) Construir el UNION ALL
selects = []
for t,y,m in tables_y:
    selects.append(select_clause("yellow", t, y, m))
for t,y,m in tables_g:
    selects.append(select_clause("green",  t, y, m))

sql_view = "CREATE OR REPLACE VIEW RAW.TRIPS_ALL AS\n" + "\nUNION ALL\n".join(selects) + ";\n"

# 6) Guardar a archivo
out = Path("create_raw_trips_all.robust.sql")
out.write_text(sql_view, encoding="utf-8")

print(f"✅ SQL robusto generado con {len(selects)} SELECTs.")
print(f"Archivo: {out.resolve()}")
print("Vista previa:")
print("\n".join(sql_view.splitlines()[:40]))

cur.close()
conn.close()


Tablas detectadas → YELLOW: 128 | GREEN: 129
✅ SQL robusto generado con 257 SELECTs.
Archivo: /home/jovyan/work/create_raw_trips_all.robust.sql
Vista previa:
CREATE OR REPLACE VIEW RAW.TRIPS_ALL AS
SELECT
  'yellow' AS service_type,
  TPEP_PICKUP_DATETIME::timestamp AS pickup_datetime,
  TPEP_DROPOFF_DATETIME::timestamp AS dropoff_datetime,
  PASSENGER_COUNT::number AS passenger_count,
  TRIP_DISTANCE::float AS trip_distance,
  PULOCATIONID::number AS pu_location_id,
  DOLOCATIONID::number AS do_location_id,
  FARE_AMOUNT::float AS fare_amount,
  TIP_AMOUNT::float AS tip_amount,
  TOLLS_AMOUNT::float AS tolls_amount,
  MTA_TAX::float AS mta_tax,
  IMPROVEMENT_SURCHARGE::float AS improvement_surcharge,
  CONGESTION_SURCHARGE::float AS congestion_surcharge,
  TOTAL_AMOUNT::float AS total_amount,
  PAYMENT_TYPE::varchar AS payment_type,
  VENDORID::varchar AS vendor_id,
  RATECODEID::varchar AS rate_code_id,
  STORE_AND_FWD_FLAG::varchar AS store_and_fwd_flag,
  2015::number  AS year,
  1

In [16]:
# N2 - CELDA 3B — Crear la VIEW con el SQL robusto ya generado

import os, snowflake.connector
from pathlib import Path

sql_path = Path("create_raw_trips_all.robust.sql")
assert sql_path.exists(), f"No encuentro {sql_path.resolve()}"

conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_HOST").replace(".snowflakecomputing.com",""),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema="RAW",
    role=os.getenv("SNOWFLAKE_ROLE"),
    client_session_keep_alive=True,
)
cur = conn.cursor()

cur.execute(f"USE DATABASE {os.getenv('SNOWFLAKE_DATABASE')}")
cur.execute("USE SCHEMA RAW")
cur.execute(f"USE WAREHOUSE {os.getenv('SNOWFLAKE_WAREHOUSE')}")
cur.execute(f"USE ROLE {os.getenv('SNOWFLAKE_ROLE')}")

sql_text = sql_path.read_text(encoding="utf-8")
cur.execute(sql_text)
print("✅ CREATE OR REPLACE VIEW RAW.TRIPS_ALL ejecutado (robusto).")

cur.execute("SHOW VIEWS LIKE 'TRIPS_ALL' IN SCHEMA RAW")
print("Vistas encontradas:", cur.fetchall())

cur.close(); conn.close()
# N2 - CELDA 3B — Crear la VIEW con el SQL robusto ya generado

import os, snowflake.connector
from pathlib import Path

sql_path = Path("create_raw_trips_all.robust.sql")
assert sql_path.exists(), f"No encuentro {sql_path.resolve()}"

conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_HOST").replace(".snowflakecomputing.com",""),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema="RAW",
    role=os.getenv("SNOWFLAKE_ROLE"),
    client_session_keep_alive=True,
)
cur = conn.cursor()

cur.execute(f"USE DATABASE {os.getenv('SNOWFLAKE_DATABASE')}")
cur.execute("USE SCHEMA RAW")
cur.execute(f"USE WAREHOUSE {os.getenv('SNOWFLAKE_WAREHOUSE')}")
cur.execute(f"USE ROLE {os.getenv('SNOWFLAKE_ROLE')}")

sql_text = sql_path.read_text(encoding="utf-8")
cur.execute(sql_text)
print("✅ CREATE OR REPLACE VIEW RAW.TRIPS_ALL ejecutado (robusto).")

cur.execute("SHOW VIEWS LIKE 'TRIPS_ALL' IN SCHEMA RAW")
print("Vistas encontradas:", cur.fetchall())

cur.close(); conn.close()


✅ CREATE OR REPLACE VIEW RAW.TRIPS_ALL ejecutado (robusto).
Vistas encontradas: [(datetime.datetime(2025, 10, 23, 8, 16, 41, 925000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'TRIPS_ALL', '', 'DM_PSET3', 'RAW', 'SYSADMIN', '', "CREATE OR REPLACE VIEW RAW.TRIPS_ALL AS\nSELECT\n  'yellow' AS service_type,\n  TPEP_PICKUP_DATETIME::timestamp AS pickup_datetime,\n  TPEP_DROPOFF_DATETIME::timestamp AS dropoff_datetime,\n  PASSENGER_COUNT::number AS passenger_count,\n  TRIP_DISTANCE::float AS trip_distance,\n  PULOCATIONID::number AS pu_location_id,\n  DOLOCATIONID::number AS do_location_id,\n  FARE_AMOUNT::float AS fare_amount,\n  TIP_AMOUNT::float AS tip_amount,\n  TOLLS_AMOUNT::float AS tolls_amount,\n  MTA_TAX::float AS mta_tax,\n  IMPROVEMENT_SURCHARGE::float AS improvement_surcharge,\n  CONGESTION_SURCHARGE::float AS congestion_surcharge,\n  TOTAL_AMOUNT::float AS total_amount,\n  PAYMENT_TYPE::varchar AS payment_type,\n  VENDORID::varchar AS vendor_id,\n  RATEC

In [20]:
# N2 - CELDA 4: verificación de la vista RAW.TRIPS_ALL

df_check = (spark.read.format("snowflake")
    .options(**sfOptions)
    .option("query", """
        SELECT
          service_type,
          year,
          COUNT(*) AS total_viajes
        FROM RAW.TRIPS_ALL
        GROUP BY service_type, year
        ORDER BY service_type, year
    """)
    .load()
)

df_check.show(50, truncate=False)


+------------+----+------------+
|SERVICE_TYPE|YEAR|TOTAL_VIAJES|
+------------+----+------------+
|green       |2015|19233765    |
|green       |2016|16385541    |
|green       |2017|11737059    |
|green       |2018|8899718     |
|green       |2019|6310985     |
|green       |2020|1734176     |
|green       |2021|1068755     |
|green       |2022|840402      |
|green       |2023|787060      |
|green       |2024|660218      |
|green       |2025|397918      |
|yellow      |2015|146039231   |
|yellow      |2016|131131805   |
|yellow      |2017|112680920   |
|yellow      |2018|102871387   |
|yellow      |2019|84597907    |
|yellow      |2020|24649092    |
|yellow      |2021|30904308    |
|yellow      |2022|39656098    |
|yellow      |2023|38310226    |
|yellow      |2024|41169720    |
|yellow      |2025|31556438    |
+------------+----+------------+

